In [ ]:
# 导入所需要的模块
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import cv2
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import models
from torchvision import transforms
from PIL import Image


In [ ]:
# 导入预训练好多VGG16网络
vgg16=models.vgg16(pretrained=True)

In [ ]:
# 读取一张图片，并对其进行可视化
im=Image.open("data/chap6/lala.jpg")
imarray=np.asarray(im)/255.0
plt.figure()
plt.imshow(imarray)
plt.show()

In [ ]:
# 将一张图像处理为VGG16网络可以处理的形式
data_transforms=transforms.Compose([
    transforms.Resize((244,244)),# 重置图像分辨率
    transforms.ToTensor(),# 转化张量并归一化处理为【0-1】
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
input_im=data_transforms(im).unsqueeze(0)
print("input_im.shape:",input_im.shape)


In [2]:
# 定义一个辅助函数，来获取指定层名称的特征
activation={}# 保存不同层的输出
def get_activation(name):
    def hook(model,input,output):
        activation[name]=output.detach()
    return hook

In [ ]:
# 获取中间的卷积后的图像特征
vgg16.eval()
# 获取网络中第四层，经过第一次最大值池化后的特征映射
vgg16.features[4].register_forward_hook(get_activation("maxpooll"))
_=vgg16(input_im)
maxpooll=activation("maxpooll.shape")


In [ ]:
# 对中间层进行可视化，可视化64个特征映射
plt.figure(figsize=(11,6))
for ii in range(maxpooll.shape[1]):
    # 可视化每张手写体
    plt.subplot(6,11,ii+1)
    plt.imshow(maxpooll.data.numpy()[0,ii,:,:],cmap="gray")
    plt.axis("off")
plt.subplots_adjust(wspace=0.1,hspace=0.1)
plt.show()

In [ ]:
# 获取VGG16模型训练的时候对于的1000个类别标签
LABEL_URL="https://s3.amazonaws.com/outcome-blog/imagenet/labels.json"
# 从网页链接中获取类别标签
response=requests.get(LABEL_URL)
label={int(key): value for key,value in response.json().items()}

In [ ]:
# 使用VGG16网络预测图像的种类
vgg16.eval()
im_pre=vgg16(input_im)
# 计算预测top-5的可能性
softmax=nn.Softmax(dim=1)
im_pre_prob=softmax(im_pre)
prob,prelab=torch.topk(im_pre_prob,5)
prob=prob.data.numpy().flatten()
prelab=prelab.numpy().flatten()
for ii,lab in enumerate(prelab):
    print("index:",lab,"label:",label[lab],"||",prob[ii])

In [ ]:
# 定义一个能够获得最后的卷积层输出和梯度的新网络
class MyVgg16(nn.Module):
    def __init__(self):
        super(MyVgg16,self).__init__()
        # 使用预训练好的VGG16模型
        self.vgg=models.vgg16(pretrained=True)
        # 切分VGG16模型，便于获取卷积层输出
        self.features_conv=self.vgg.features[:30]
        # 使用原始的最大池化层
        self.max_pool=self.vgg.features[30]
        self.avgpool=self.vgg.avgpool
        # 使用VGG16的分类层
        self.classifier=self.vgg.classifier
        # 生成梯度占位符
        self.gradients=None
    # 获取梯度的钩子函数
    def activation_hook(self,grad):
        self.gradients=grad
    def forward(self,x):
        x=self.features_conv(x)
        # 注册钩子
        h=x.register_hook(self.activation_hook)
        # 对卷积后的输出使用最大值池化
        x=self.max_pool(x)
        x=self.avgpool(x)
        x=x.view((1,-1))
        x=self.classifier(x)
        return x
    # 获取梯度的方法
    def get_activation_gradient(self):
        return self.gradients
    # 获取卷积层输出的方法
    def get_activation(self,x):
        return self.features_conv(x)
        

In [ ]:
# 初始化网络
vggcam=MyVgg16()
# 设置网络的模式
vggcam.eval()
# 计算网络对于图像的预测值
im_pre=vggcam(input_im)
# 计算预测top-5的可能性
softmax=nn.Softmax(dim=1)
im_pre_prob=softmax(im_pre)
prob,prelab=torch.topk(im_pre_prob,5)
prob=prob.data.numpy().flatten()
prelab=prelab.numpy().flatten()
for ii,lab in enumerate(prelab):
    print("index:",lab,"label:",label[lab],"||",prob[ii])

In [ ]:
# 获取相对于模型参数的输出梯度
im_pre[:,prelab[0]].backward()
# 获取模型的梯度
gradients=vggcam.get_activation_gradient()
# 计算梯度相应通道的均值
mean_gradients=torch.mean(gradients,dim=[0,2,3])
# 获取图像在相应卷积层输出的卷积特征
activations=vggcam.get_activation(input_im).detach()
# 每个通道乘以相应的梯度均值
for i in range(len(mean_gradients)):
    activations[:,i,:,:]*=mean_gradients[i]
# 计算所有通道的均值输出得到热力图
heatmap=torch.mean(activations,dim=1),squeeze()
# 使用relu激活函数作用于生成热力图
heatmap=F.relu(heatmap)
# 对热力图进行标准化
heatmap/=torch.max(heatmap)
heatmap=heatmap.numpy()
# 可视化热力图
plt.matshow(heatmap)


In [ ]:
# 将CAM热力图融合到原始图像上
img=cv2.imread("data/chap6/lala.jpg")
heatmap=cv2.resize(heatmap,(img.shape[1],img.shape[0]))
heatmap=np.unit8(255*heatmap)
heatmap=cv2.applyColorMap(heatmap,cv2.COLORMAP_JET)
Grad_cam_img=heatmap*0.4+img
Grad_cam_img=Grad_cam_img/Grad_cam_img.max()
# 可视化图像
b,g,r=cv2.split(Grad_cam_img)
Grad_cam_img=cv2.merge([r,g,b])
plt.figure()
plt.imshow(Grad_cam_img)
plt.show()